# Production RAG Implementation Guide

This notebook contains all code referenced in the Production RAG Bible.

**Setup**: Run the installation cell, then jump to the pattern you need.

**Structure**:
1. Installation & Setup
2. Evaluation Framework
3. RAG Pattern Implementations
4. Benchmarking Tools
5. Production Utilities

## 1. Installation & Setup

In [ ]:
# Core dependencies
!pip install -q openai anthropic
!pip install -q langchain langchain-community langchain-openai
!pip install -q chromadb sentence-transformers
!pip install -q numpy pandas scikit-learn
!pip install -q tqdm python-dotenv
!pip install -q ragas  # For eval metrics

# Optional: For GraphRAG
# !pip install -q neo4j spacy
# !python -m spacy download en_core_web_sm

# Optional: For reranking
# !pip install -q cohere voyageai

In [ ]:
import os
from typing import List, Dict, Tuple, Optional
import numpy as np
import pandas as pd
from dataclasses import dataclass
import time
from collections import defaultdict
import json

# LLM clients
from openai import OpenAI
import anthropic

# Vector store
import chromadb
from chromadb.utils import embedding_functions

# Embeddings
from sentence_transformers import SentenceTransformer

# Set your API keys
os.environ["OPENAI_API_KEY"] = "your-api-key-here"
os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

# Initialize clients
openai_client = OpenAI()
anthropic_client = anthropic.Anthropic()

## 2. Evaluation Framework

**Critical**: Build this first. Every pattern depends on measurement.

In [ ]:
@dataclass
class EvalExample:
    """Single evaluation example"""
    query: str
    ground_truth_answer: str
    relevant_doc_ids: List[str]  # IDs of documents that should be retrieved
    category: str = "general"  # For segmented analysis

@dataclass
class RetrievalMetrics:
    """Retrieval evaluation results"""
    precision_at_k: Dict[int, float]  # {5: 0.6, 10: 0.5}
    recall_at_k: Dict[int, float]
    mrr: float
    num_queries: int

@dataclass
class AnswerMetrics:
    """Answer evaluation results"""
    correctness: float
    faithfulness: float
    relevance: float
    num_queries: int

@dataclass
class LatencyMetrics:
    """Latency measurements"""
    p50: float
    p95: float
    p99: float
    mean: float

In [ ]:
class RetrievalEvaluator:
    """Evaluate retrieval quality"""
    
    @staticmethod
    def precision_at_k(retrieved_ids: List[str], 
                       relevant_ids: List[str], 
                       k: int) -> float:
        """Precision@K: What % of top-K retrieved docs are relevant?"""
        if k == 0:
            return 0.0
        
        retrieved_at_k = set(retrieved_ids[:k])
        relevant_set = set(relevant_ids)
        
        num_relevant = len(retrieved_at_k.intersection(relevant_set))
        return num_relevant / k
    
    @staticmethod
    def recall_at_k(retrieved_ids: List[str], 
                    relevant_ids: List[str], 
                    k: int) -> float:
        """Recall@K: What % of all relevant docs are in top-K?"""
        if len(relevant_ids) == 0:
            return 0.0
        
        retrieved_at_k = set(retrieved_ids[:k])
        relevant_set = set(relevant_ids)
        
        num_found = len(retrieved_at_k.intersection(relevant_set))
        return num_found / len(relevant_ids)
    
    @staticmethod
    def reciprocal_rank(retrieved_ids: List[str], 
                       relevant_ids: List[str]) -> float:
        """Reciprocal Rank: 1 / position of first relevant doc"""
        relevant_set = set(relevant_ids)
        
        for idx, doc_id in enumerate(retrieved_ids, 1):
            if doc_id in relevant_set:
                return 1.0 / idx
        
        return 0.0
    
    def evaluate(self, 
                 eval_examples: List[EvalExample],
                 retrieval_fn,
                 k_values: List[int] = [5, 10]) -> RetrievalMetrics:
        """Run full retrieval evaluation"""
        
        precision_scores = {k: [] for k in k_values}
        recall_scores = {k: [] for k in k_values}
        rr_scores = []
        
        for example in eval_examples:
            # Get retrieved doc IDs from your system
            retrieved_ids = retrieval_fn(example.query)
            
            # Calculate metrics
            for k in k_values:
                p = self.precision_at_k(retrieved_ids, example.relevant_doc_ids, k)
                r = self.recall_at_k(retrieved_ids, example.relevant_doc_ids, k)
                precision_scores[k].append(p)
                recall_scores[k].append(r)
            
            rr = self.reciprocal_rank(retrieved_ids, example.relevant_doc_ids)
            rr_scores.append(rr)
        
        return RetrievalMetrics(
            precision_at_k={k: np.mean(scores) for k, scores in precision_scores.items()},
            recall_at_k={k: np.mean(scores) for k, scores in recall_scores.items()},
            mrr=np.mean(rr_scores),
            num_queries=len(eval_examples)
        )

# Usage example
# evaluator = RetrievalEvaluator()
# metrics = evaluator.evaluate(eval_examples, my_retrieval_function)
# print(f"Precision@5: {metrics.precision_at_k[5]:.3f}")
# print(f"Recall@10: {metrics.recall_at_k[10]:.3f}")
# print(f"MRR: {metrics.mrr:.3f}")

In [ ]:
class AnswerEvaluator:
    """Evaluate answer quality using LLM-as-Judge"""
    
    def __init__(self, client):
        self.client = client
    
    def evaluate_correctness(self, 
                            query: str, 
                            generated_answer: str, 
                            ground_truth: str) -> float:
        """Score: Does the answer match ground truth?"""
        
        prompt = f"""You are evaluating answer correctness.

Query: {query}
Ground Truth Answer: {ground_truth}
Generated Answer: {generated_answer}

Does the generated answer convey the same information as the ground truth?
Respond with a score from 0.0 to 1.0, where:
- 1.0 = Perfect match (same facts, same meaning)
- 0.7-0.9 = Mostly correct (minor differences)
- 0.4-0.6 = Partially correct (missing key info)
- 0.0-0.3 = Incorrect or contradictory

Respond with ONLY a number between 0.0 and 1.0."""
        
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=10,
            messages=[{"role": "user", "content": prompt}]
        )
        
        try:
            score = float(response.content[0].text.strip())
            return max(0.0, min(1.0, score))
        except:
            return 0.0
    
    def evaluate_faithfulness(self, 
                             generated_answer: str, 
                             retrieved_docs: List[str]) -> float:
        """Score: Are all claims in the answer supported by the docs?"""
        
        context = "\n\n".join(retrieved_docs)
        
        prompt = f"""You are evaluating answer faithfulness.

Retrieved Context:
{context}

Generated Answer:
{generated_answer}

Are ALL claims in the answer supported by the context?
Respond with a score from 0.0 to 1.0, where:
- 1.0 = Every claim has support in context
- 0.7-0.9 = Most claims supported (minor unsupported details)
- 0.4-0.6 = Some claims unsupported
- 0.0-0.3 = Many claims unsupported or contradictory

Respond with ONLY a number between 0.0 and 1.0."""
        
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=10,
            messages=[{"role": "user", "content": prompt}]
        )
        
        try:
            score = float(response.content[0].text.strip())
            return max(0.0, min(1.0, score))
        except:
            return 0.0
    
    def evaluate_relevance(self, 
                          query: str, 
                          generated_answer: str) -> float:
        """Score: Does the answer actually address the question?"""
        
        prompt = f"""You are evaluating answer relevance.

Query: {query}
Generated Answer: {generated_answer}

Does the answer directly address the question asked?
Respond with a score from 0.0 to 1.0, where:
- 1.0 = Perfectly addresses the question
- 0.7-0.9 = Mostly relevant (minor tangents)
- 0.4-0.6 = Partially relevant
- 0.0-0.3 = Off-topic or doesn't answer the question

Respond with ONLY a number between 0.0 and 1.0."""
        
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=10,
            messages=[{"role": "user", "content": prompt}]
        )
        
        try:
            score = float(response.content[0].text.strip())
            return max(0.0, min(1.0, score))
        except:
            return 0.0
    
    def evaluate_batch(self, 
                      eval_examples: List[EvalExample],
                      rag_fn) -> AnswerMetrics:
        """Run full answer evaluation"""
        
        correctness_scores = []
        faithfulness_scores = []
        relevance_scores = []
        
        for example in eval_examples:
            # Generate answer using your RAG system
            answer, retrieved_docs = rag_fn(example.query)
            
            # Evaluate
            c = self.evaluate_correctness(example.query, answer, example.ground_truth_answer)
            f = self.evaluate_faithfulness(answer, retrieved_docs)
            r = self.evaluate_relevance(example.query, answer)
            
            correctness_scores.append(c)
            faithfulness_scores.append(f)
            relevance_scores.append(r)
        
        return AnswerMetrics(
            correctness=np.mean(correctness_scores),
            faithfulness=np.mean(faithfulness_scores),
            relevance=np.mean(relevance_scores),
            num_queries=len(eval_examples)
        )

# Usage example
# evaluator = AnswerEvaluator(anthropic_client)
# metrics = evaluator.evaluate_batch(eval_examples, my_rag_function)
# print(f"Correctness: {metrics.correctness:.3f}")
# print(f"Faithfulness: {metrics.faithfulness:.3f}")

In [ ]:
class LatencyBenchmark:
    """Measure latency distribution"""
    
    @staticmethod
    def measure(fn, queries: List[str], warmup: int = 3) -> LatencyMetrics:
        """Measure latency for a function"""
        
        # Warmup
        for i in range(min(warmup, len(queries))):
            fn(queries[i])
        
        # Actual measurement
        latencies = []
        for query in queries:
            start = time.time()
            fn(query)
            latency = time.time() - start
            latencies.append(latency * 1000)  # Convert to ms
        
        latencies = np.array(latencies)
        
        return LatencyMetrics(
            p50=np.percentile(latencies, 50),
            p95=np.percentile(latencies, 95),
            p99=np.percentile(latencies, 99),
            mean=np.mean(latencies)
        )

# Usage example
# latency = LatencyBenchmark.measure(my_rag_function, test_queries)
# print(f"P95: {latency.p95:.0f}ms")

## 3. Synthetic Test Data Generation

In [ ]:
class SyntheticEvalGenerator:
    """Generate evaluation examples from documents"""
    
    def __init__(self, client):
        self.client = client
    
    def generate_questions(self, 
                          document: str, 
                          doc_id: str,
                          num_questions: int = 5) -> List[EvalExample]:
        """Generate questions from a document"""
        
        prompt = f"""You are generating test questions from a document.

Document:
{document}

Generate {num_questions} questions that can be answered using ONLY this document.
Include a mix of:
- Simple factual questions (Who, What, When)
- Reasoning questions (Why, How)
- Questions about specific details

For each question, provide:
1. The question
2. The answer (based on the document)

Format as JSON:
{{
  "questions": [
    {{
      "question": "What is...",
      "answer": "According to the document..."
    }}
  ]
}}"""
        
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            messages=[{"role": "user", "content": prompt}]
        )
        
        try:
            data = json.loads(response.content[0].text)
            examples = []
            
            for item in data["questions"]:
                examples.append(EvalExample(
                    query=item["question"],
                    ground_truth_answer=item["answer"],
                    relevant_doc_ids=[doc_id]
                ))
            
            return examples
        except:
            return []
    
    def generate_eval_set(self, 
                         documents: List[Tuple[str, str]],  # (doc_id, text)
                         questions_per_doc: int = 3) -> List[EvalExample]:
        """Generate full eval set from documents"""
        
        all_examples = []
        
        for doc_id, text in documents:
            examples = self.generate_questions(text, doc_id, questions_per_doc)
            all_examples.extend(examples)
        
        return all_examples

# Usage example
# generator = SyntheticEvalGenerator(anthropic_client)
# eval_set = generator.generate_eval_set(my_documents)
# print(f"Generated {len(eval_set)} evaluation examples")

## 4. RAG Pattern Implementations

### Pattern 1: Naive RAG (Baseline)

In [ ]:
class NaiveRAG:
    """Basic vector search + context stuffing"""
    
    def __init__(self, collection_name: str = "naive_rag"):
        # Initialize vector DB
        self.chroma_client = chromadb.Client()
        
        # Use OpenAI embeddings
        self.embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
            api_key=os.environ["OPENAI_API_KEY"],
            model_name="text-embedding-3-small"
        )
        
        # Create collection
        self.collection = self.chroma_client.create_collection(
            name=collection_name,
            embedding_function=self.embedding_fn,
            metadata={"hnsw:space": "cosine"}
        )
        
        self.llm_client = openai_client
    
    def index_documents(self, documents: List[Dict[str, str]]):
        """Index documents into vector DB
        
        documents: List of {"id": str, "text": str, "metadata": dict}
        """
        ids = [doc["id"] for doc in documents]
        texts = [doc["text"] for doc in documents]
        metadatas = [doc.get("metadata", {}) for doc in documents]
        
        self.collection.add(
            ids=ids,
            documents=texts,
            metadatas=metadatas
        )
    
    def retrieve(self, query: str, top_k: int = 5) -> Tuple[List[str], List[str]]:
        """Retrieve relevant documents
        
        Returns: (doc_ids, doc_texts)
        """
        results = self.collection.query(
            query_texts=[query],
            n_results=top_k
        )
        
        doc_ids = results["ids"][0]
        doc_texts = results["documents"][0]
        
        return doc_ids, doc_texts
    
    def generate(self, query: str, context: List[str]) -> str:
        """Generate answer from context"""
        
        context_str = "\n\n".join([
            f"[Document {i+1}]\n{doc}" 
            for i, doc in enumerate(context)
        ])
        
        prompt = f"""You are a helpful assistant. Answer the question based on the context provided.

Context:
{context_str}

Question: {query}

Answer:"""
        
        response = self.llm_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        
        return response.choices[0].message.content
    
    def query(self, query: str, top_k: int = 5) -> Tuple[str, List[str]]:
        """End-to-end RAG pipeline
        
        Returns: (answer, retrieved_docs)
        """
        # Retrieve
        doc_ids, doc_texts = self.retrieve(query, top_k)
        
        # Generate
        answer = self.generate(query, doc_texts)
        
        return answer, doc_texts

# Usage example
# rag = NaiveRAG()
# rag.index_documents(my_documents)
# answer, docs = rag.query("What is...?")
# print(answer)

### Pattern 2: Parent Document Retrieval

In [ ]:
class ParentDocumentRAG:
    """Search small chunks, return full parent documents"""
    
    def __init__(self, 
                 collection_name: str = "parent_doc_rag",
                 chunk_size: int = 256,
                 chunk_overlap: int = 50):
        
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        
        # Vector DB for chunks
        self.chroma_client = chromadb.Client()
        self.embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
            api_key=os.environ["OPENAI_API_KEY"],
            model_name="text-embedding-3-small"
        )
        self.collection = self.chroma_client.create_collection(
            name=collection_name,
            embedding_function=self.embedding_fn
        )
        
        # Store parent documents
        self.parent_docs = {}
        
        self.llm_client = openai_client
    
    def chunk_text(self, text: str, parent_id: str) -> List[Dict]:
        """Split text into overlapping chunks"""
        chunks = []
        words = text.split()
        
        for i in range(0, len(words), self.chunk_size - self.chunk_overlap):
            chunk_words = words[i:i + self.chunk_size]
            chunk_text = " ".join(chunk_words)
            
            chunks.append({
                "id": f"{parent_id}_chunk_{len(chunks)}",
                "text": chunk_text,
                "parent_id": parent_id
            })
        
        return chunks
    
    def index_documents(self, documents: List[Dict[str, str]]):
        """Index documents with chunking
        
        documents: List of {"id": str, "text": str}
        """
        all_chunks = []
        
        for doc in documents:
            # Store parent
            self.parent_docs[doc["id"]] = doc["text"]
            
            # Create chunks
            chunks = self.chunk_text(doc["text"], doc["id"])
            all_chunks.extend(chunks)
        
        # Index chunks
        if all_chunks:
            self.collection.add(
                ids=[c["id"] for c in all_chunks],
                documents=[c["text"] for c in all_chunks],
                metadatas=[{"parent_id": c["parent_id"]} for c in all_chunks]
            )
    
    def retrieve(self, query: str, top_k: int = 10) -> List[str]:
        """Retrieve parent documents based on chunk search"""
        
        # Search chunks
        results = self.collection.query(
            query_texts=[query],
            n_results=top_k
        )
        
        # Get unique parent IDs
        parent_ids = []
        seen = set()
        
        for metadata in results["metadatas"][0]:
            parent_id = metadata["parent_id"]
            if parent_id not in seen:
                parent_ids.append(parent_id)
                seen.add(parent_id)
        
        # Return full parent documents
        parent_texts = [self.parent_docs[pid] for pid in parent_ids]
        
        return parent_texts[:5]  # Limit to top 5 parents
    
    def query(self, query: str) -> Tuple[str, List[str]]:
        """End-to-end RAG pipeline"""
        
        # Retrieve parent docs
        parent_docs = self.retrieve(query)
        
        # Generate answer
        context_str = "\n\n".join([
            f"[Document {i+1}]\n{doc}" 
            for i, doc in enumerate(parent_docs)
        ])
        
        prompt = f"""Answer the question based on the context.

Context:
{context_str}

Question: {query}

Answer:"""
        
        response = self.llm_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        
        answer = response.choices[0].message.content
        
        return answer, parent_docs

# Usage example
# rag = ParentDocumentRAG(chunk_size=256)
# rag.index_documents(my_documents)
# answer, docs = rag.query("What is...?")

### Pattern 3: Corrective RAG (CRAG)

In [ ]:
class CorrectiveRAG:
    """RAG with relevance grading and fallback"""
    
    def __init__(self, 
                 collection_name: str = "crag",
                 relevance_threshold: float = 0.6):
        
        self.relevance_threshold = relevance_threshold
        
        # Base RAG
        self.base_rag = NaiveRAG(collection_name)
        self.llm_client = anthropic_client
    
    def index_documents(self, documents: List[Dict[str, str]]):
        """Index documents"""
        self.base_rag.index_documents(documents)
    
    def grade_document(self, query: str, document: str) -> float:
        """Grade document relevance to query
        
        Returns: Relevance score 0.0-1.0
        """
        
        prompt = f"""You are a document grader.

Query: {query}

Document:
{document}

Is this document relevant to answering the query?
Respond with ONLY a score from 0.0 to 1.0 where:
- 1.0 = Highly relevant, contains answer
- 0.5-0.8 = Somewhat relevant
- 0.0-0.4 = Not relevant

Score:"""
        
        response = self.llm_client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=10,
            messages=[{"role": "user", "content": prompt}]
        )
        
        try:
            score = float(response.content[0].text.strip())
            return max(0.0, min(1.0, score))
        except:
            return 0.0
    
    def web_search_fallback(self, query: str) -> str:
        """Fallback to web search
        
        In production, integrate with real web search API
        """
        return f"[Web search would be triggered for: {query}]"
    
    def query(self, query: str, top_k: int = 10) -> Tuple[str, List[str]]:
        """CRAG pipeline with grading and fallback"""
        
        # Initial retrieval
        doc_ids, doc_texts = self.base_rag.retrieve(query, top_k)
        
        # Grade each document
        graded_docs = []
        for doc in doc_texts:
            score = self.grade_document(query, doc)
            if score >= self.relevance_threshold:
                graded_docs.append(doc)
        
        # Check if we have enough relevant docs
        if len(graded_docs) < 2:
            # Trigger fallback
            fallback_result = self.web_search_fallback(query)
            graded_docs.append(fallback_result)
        
        # Generate answer from verified docs
        if graded_docs:
            answer = self.base_rag.generate(query, graded_docs)
        else:
            answer = "I don't have enough reliable information to answer this question."
        
        return answer, graded_docs

# Usage example
# rag = CorrectiveRAG(relevance_threshold=0.7)
# rag.index_documents(my_documents)
# answer, verified_docs = rag.query("What is...?")

### Pattern 4: HyDE (Hypothetical Document Embeddings)

In [ ]:
class HyDERAG:
    """Generate hypothetical answer, search with it"""
    
    def __init__(self, collection_name: str = "hyde_rag"):
        self.base_rag = NaiveRAG(collection_name)
        self.llm_client = openai_client
        
        # Cache for common queries
        self.hyde_cache = {}
    
    def index_documents(self, documents: List[Dict[str, str]]):
        """Index documents"""
        self.base_rag.index_documents(documents)
    
    def generate_hypothetical_doc(self, query: str) -> str:
        """Generate hypothetical answer to query"""
        
        # Check cache
        if query in self.hyde_cache:
            return self.hyde_cache[query]
        
        prompt = f"""Write a detailed, factual answer to this question as if you were a technical document.
Use formal language and technical vocabulary.

Question: {query}

Answer:"""
        
        response = self.llm_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        
        hypothetical_doc = response.choices[0].message.content
        
        # Cache it
        self.hyde_cache[query] = hypothetical_doc
        
        return hypothetical_doc
    
    def query(self, query: str, top_k: int = 5) -> Tuple[str, List[str]]:
        """HyDE pipeline"""
        
        # Generate hypothetical document
        hypothetical_doc = self.generate_hypothetical_doc(query)
        
        # Search using hypothetical doc
        doc_ids, doc_texts = self.base_rag.retrieve(hypothetical_doc, top_k)
        
        # Generate real answer from real docs
        answer = self.base_rag.generate(query, doc_texts)
        
        return answer, doc_texts

# Usage example
# rag = HyDERAG()
# rag.index_documents(my_documents)
# answer, docs = rag.query("How do I...?")

### Pattern 5: Reranking

In [ ]:
class RerankedRAG:
    """Vector search + cross-encoder reranking"""
    
    def __init__(self, 
                 collection_name: str = "reranked_rag",
                 reranker_model: str = "cross-encoder/ms-marco-MiniLM-L-6-v2"):
        
        self.base_rag = NaiveRAG(collection_name)
        
        # Load reranker model
        from sentence_transformers import CrossEncoder
        self.reranker = CrossEncoder(reranker_model)
    
    def index_documents(self, documents: List[Dict[str, str]]):
        """Index documents"""
        self.base_rag.index_documents(documents)
    
    def rerank(self, 
               query: str, 
               documents: List[str], 
               top_k: int = 5) -> List[str]:
        """Rerank documents using cross-encoder"""
        
        # Score each query-doc pair
        pairs = [[query, doc] for doc in documents]
        scores = self.reranker.predict(pairs)
        
        # Sort by score
        doc_scores = list(zip(documents, scores))
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Return top k
        reranked_docs = [doc for doc, score in doc_scores[:top_k]]
        
        return reranked_docs
    
    def query(self, 
              query: str, 
              retrieval_k: int = 50,
              final_k: int = 5) -> Tuple[str, List[str]]:
        """Retrieve many, rerank to few"""
        
        # Retrieve many candidates
        doc_ids, doc_texts = self.base_rag.retrieve(query, retrieval_k)
        
        # Rerank to top k
        reranked_docs = self.rerank(query, doc_texts, final_k)
        
        # Generate answer
        answer = self.base_rag.generate(query, reranked_docs)
        
        return answer, reranked_docs

# Usage example
# rag = RerankedRAG()
# rag.index_documents(my_documents)
# answer, docs = rag.query("What is...?", retrieval_k=100, final_k=5)

## 5. Complete Evaluation Pipeline

In [ ]:
def run_full_evaluation(rag_system, 
                       eval_examples: List[EvalExample],
                       name: str = "RAG System"):
    """Run complete evaluation suite"""
    
    print(f"\n{'='*60}")
    print(f"Evaluating: {name}")
    print(f"{'='*60}\n")
    
    # 1. Retrieval Evaluation
    print("[1/3] Retrieval Evaluation...")
    
    def retrieval_fn(query):
        doc_ids, _ = rag_system.retrieve(query, top_k=10)
        return doc_ids
    
    retrieval_eval = RetrievalEvaluator()
    retrieval_metrics = retrieval_eval.evaluate(eval_examples, retrieval_fn)
    
    print(f"  Precision@5:  {retrieval_metrics.precision_at_k[5]:.3f}")
    print(f"  Recall@10:    {retrieval_metrics.recall_at_k[10]:.3f}")
    print(f"  MRR:          {retrieval_metrics.mrr:.3f}")
    
    # 2. Answer Evaluation (sample for speed)
    print("\n[2/3] Answer Evaluation (sample of 20)...")
    
    sample_examples = eval_examples[:20]
    
    def rag_fn(query):
        return rag_system.query(query)
    
    answer_eval = AnswerEvaluator(anthropic_client)
    answer_metrics = answer_eval.evaluate_batch(sample_examples, rag_fn)
    
    print(f"  Correctness:  {answer_metrics.correctness:.3f}")
    print(f"  Faithfulness: {answer_metrics.faithfulness:.3f}")
    print(f"  Relevance:    {answer_metrics.relevance:.3f}")
    
    # 3. Latency Benchmark
    print("\n[3/3] Latency Benchmark...")
    
    test_queries = [ex.query for ex in eval_examples[:30]]
    
    def query_fn(q):
        return rag_system.query(q)
    
    latency_metrics = LatencyBenchmark.measure(query_fn, test_queries)
    
    print(f"  P50:  {latency_metrics.p50:.0f}ms")
    print(f"  P95:  {latency_metrics.p95:.0f}ms")
    print(f"  P99:  {latency_metrics.p99:.0f}ms")
    
    # Summary
    print(f"\n{'='*60}")
    print("Summary")
    print(f"{'='*60}")
    
    # Pass/Fail checks
    checks = [
        ("Precision@5 > 0.6", retrieval_metrics.precision_at_k[5] > 0.6),
        ("Recall@10 > 0.8", retrieval_metrics.recall_at_k[10] > 0.8),
        ("MRR > 0.5", retrieval_metrics.mrr > 0.5),
        ("Correctness > 0.85", answer_metrics.correctness > 0.85),
        ("Faithfulness > 0.9", answer_metrics.faithfulness > 0.9),
        ("P95 Latency < 3000ms", latency_metrics.p95 < 3000),
    ]
    
    for check_name, passed in checks:
        status = "✓ PASS" if passed else "✗ FAIL"
        print(f"  {status}  {check_name}")
    
    all_passed = all(passed for _, passed in checks)
    
    if all_passed:
        print("\n🎉 All checks passed! System ready for production.")
    else:
        print("\n⚠️  Some checks failed. Review metrics before deploying.")
    
    print(f"\n{'='*60}\n")

# Usage example
# my_rag = NaiveRAG()
# my_rag.index_documents(documents)
# run_full_evaluation(my_rag, eval_examples, "Naive RAG")

## 6. Example: End-to-End Workflow

In [ ]:
# Example workflow

# Step 1: Prepare documents
sample_documents = [
    {
        "id": "doc_1",
        "text": "Python is a high-level programming language. It was created by Guido van Rossum and released in 1991."
    },
    {
        "id": "doc_2",
        "text": "Machine learning is a subset of artificial intelligence. It focuses on building systems that learn from data."
    },
    {
        "id": "doc_3",
        "text": "RAG (Retrieval Augmented Generation) combines retrieval systems with language models to provide grounded answers."
    },
]

# Step 2: Generate eval set
generator = SyntheticEvalGenerator(anthropic_client)
eval_examples = generator.generate_eval_set(
    [(doc["id"], doc["text"]) for doc in sample_documents],
    questions_per_doc=2
)

print(f"Generated {len(eval_examples)} eval examples")

# Step 3: Test multiple RAG patterns
patterns = [
    ("Naive RAG", NaiveRAG("test_naive")),
    ("Parent Doc RAG", ParentDocumentRAG("test_parent")),
    ("CRAG", CorrectiveRAG("test_crag")),
]

for name, rag_system in patterns:
    # Index documents
    rag_system.index_documents(sample_documents)
    
    # Run evaluation
    run_full_evaluation(rag_system, eval_examples, name)

# Step 4: Compare and choose
print("\nCompare the results above to choose the best pattern for your use case.")

## 7. Production Monitoring Setup

In [ ]:
class ProductionMonitor:
    """Track production metrics"""
    
    def __init__(self):
        self.latencies = []
        self.errors = []
        self.null_responses = 0
        self.total_queries = 0
        self.cache_hits = 0
        self.user_feedback = {"positive": 0, "negative": 0}
    
    def log_query(self, 
                  latency_ms: float, 
                  was_cached: bool = False,
                  was_null: bool = False,
                  error: Optional[str] = None):
        """Log a single query"""
        
        self.total_queries += 1
        self.latencies.append(latency_ms)
        
        if was_cached:
            self.cache_hits += 1
        
        if was_null:
            self.null_responses += 1
        
        if error:
            self.errors.append(error)
    
    def log_feedback(self, is_positive: bool):
        """Log user feedback"""
        if is_positive:
            self.user_feedback["positive"] += 1
        else:
            self.user_feedback["negative"] += 1
    
    def get_metrics(self) -> Dict:
        """Get current metrics"""
        
        if not self.latencies:
            return {}
        
        latencies = np.array(self.latencies)
        
        total_feedback = sum(self.user_feedback.values())
        satisfaction = self.user_feedback["positive"] / total_feedback if total_feedback > 0 else 0
        
        return {
            "total_queries": self.total_queries,
            "error_rate": len(self.errors) / self.total_queries,
            "null_response_rate": self.null_responses / self.total_queries,
            "cache_hit_rate": self.cache_hits / self.total_queries,
            "latency_p50": np.percentile(latencies, 50),
            "latency_p95": np.percentile(latencies, 95),
            "latency_p99": np.percentile(latencies, 99),
            "user_satisfaction": satisfaction,
        }
    
    def check_alerts(self) -> List[str]:
        """Check if any alert thresholds are breached"""
        
        alerts = []
        metrics = self.get_metrics()
        
        if metrics.get("error_rate", 0) > 0.05:
            alerts.append("🚨 CRITICAL: Error rate > 5%")
        
        if metrics.get("latency_p95", 0) > 5000:
            alerts.append("🚨 CRITICAL: P95 latency > 5s")
        
        if metrics.get("null_response_rate", 0) > 0.10:
            alerts.append("⚠️  WARNING: Null response rate > 10%")
        
        if metrics.get("cache_hit_rate", 0) < 0.20:
            alerts.append("⚠️  WARNING: Cache hit rate < 20%")
        
        return alerts

# Usage example
# monitor = ProductionMonitor()
# 
# # In your RAG endpoint:
# start = time.time()
# answer, docs = rag.query(user_query)
# latency = (time.time() - start) * 1000
# 
# monitor.log_query(
#     latency_ms=latency,
#     was_null=(answer == "I don't know")
# )
# 
# # Check for alerts
# alerts = monitor.check_alerts()
# for alert in alerts:
#     print(alert)

## 8. Quick Start Guide

**For first-time users**: Start here and follow the steps.

```python
# 1. Prepare your data
documents = [
    {"id": "doc_1", "text": "Your first document..."},
    {"id": "doc_2", "text": "Your second document..."},
]

# 2. Start with Naive RAG
rag = NaiveRAG()
rag.index_documents(documents)

# 3. Generate eval set
generator = SyntheticEvalGenerator(anthropic_client)
eval_set = generator.generate_eval_set(
    [(d["id"], d["text"]) for d in documents]
)

# 4. Run evaluation
run_full_evaluation(rag, eval_set, "Naive RAG")

# 5. If metrics fail:
# - Precision@5 < 0.6 → Try ParentDocumentRAG or tune chunk sizes
# - Faithfulness < 0.9 → Try CorrectiveRAG
# - Recall@10 < 0.8 → Try HyDE or better chunking
# - MRR < 0.5 → Add reranking

# 6. Deploy with monitoring
monitor = ProductionMonitor()
# ... integrate into your API
```